# This notebook is used to plot figure 1 of epistasis paper by randomly sampling a gene sets from pairs for each gene

In [6]:
%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm
import os
import gc
from scipy.stats.stats import pearsonr
from itertools import combinations
import random
import multiprocessing
import concurrent.futures
import string
random.seed(1234)

## IntAct

In [7]:
inpath_intact = '../mutex_data/'
intact_edge_file = inpath_intact+'intact_nodupl_edge_file.txt'
intact_index_file = inpath_intact+'intact_nodupl_index_file.txt'

with open(intact_index_file, 'r') as f:
    indices = {line.split()[0]:line.split()[1] for line in f.readlines()}

with open(intact_edge_file, 'r') as f:
    edges = [(indices[line.split()[0]].upper(),indices[line.split()[1]].upper()) for line in f.readlines()]
len(edges)

intact_genes_list = list(indices.values())
intact_genes_list[:5]

['""CHEBI', '100147744', '1B', '1EFV', '1KLA']

## COSMIC

In [8]:
inpath_cosmic = '../mutex_data/'
cosmic_infile = inpath_cosmic+'Census_allFri_Apr_26_12_49_57_2019.tsv'
with open(cosmic_infile,'r') as f:
    cosmic_genes = [line.split()[0].upper() for line in f.readlines()[1:]]
print(len(cosmic_genes))
# cosmic_genes

723


## Functions

In [13]:
def set_input_file_paths(methods, c, t):
    '''set input paths as per the location of the MEX results
    m:methods, c: cancer type, t: threshold'''
    
    strat_list=['COADREAD']
    dict_infile = {}
    dict_infile_intact = {}
    for m in methods:
        if m == 'discover':
            suffix = '{}_mutation_filtered_ep_data/{}_{}_result_mutations_all_genes_q1.0_normal_{}.txt'.format(m,c,m,t)
            suffix_intact = '{}_mutation_filtered_ep_data/{}_pairs_q1.0_normal_intact_filtered_subset{}.txt'.format(m,c,t)
        elif m == 'discover_strat':
            suffix = '{}_mutation_filtered_ep_data/{}_{}_result_mutations_all_genes_q1.0_stratified_{}.txt'.format('discover',c,'discover',t)
            suffix_intact = '{}_mutation_filtered_ep_data/{}_pairs_q1.0_stratified_intact_filtered_subset{}.txt'.format('discover',c,t)    
        else:
            suffix = '{}_mutation_filtered_ep_data/{}_{}_result_mutations_all_genes_{}.txt'.format(m,c,m,t)
            suffix_intact = '{}_mutation_filtered_ep_data/{}_{}_pairs_intact_filtered_subset{}.txt'.format(m,c,m,t)

        dict_infile[m] = '../mutex_data/' + suffix
        dict_infile_intact[m] = '../mutex_data/' + suffix_intact
    
    return dict_infile, dict_infile_intact

def load_cgcg_pairs(methods,dict_infile, ref_genes=cosmic_genes):
    '''
    load mutual exclusivity values for CGC-CGC pairs
    methods: mutual exclusivity methods, ref_genes=reference driver genes (COSMIC)
    '''
    d_out ={}
    for m in tqdm(methods):
        d_nb={}
        infile=dict_infile[m]
        with open(infile) as f:
            for line in tqdm(f.readlines()):
                line=line.split()
                g1=line[1]
                g2=line[2]
                if g1 in ref_genes and g2 in ref_genes:
                    if g1 not in d_nb:
                        d_nb[g1]={}
                    if g2 not in d_nb:
                        d_nb[g2]={}
                    d_nb[g1][g2]=float(line[3])
                    d_nb[g2][g1]=float(line[3])
        d_out[m]=d_nb

    return d_out

def get_sig_dict(d,m):
    '''get percentage significance from a dictionary of CGC-CGC neighbors
    '''
    d_out={}
    for g in d:
        count=0
        for k,v in d[g].items():
            if m=='wext' and v==0:
                continue
            elif v<0.05:
                count+=1
        d_out[g]= float(count)/float(len(d[g]))
    
    return d_out
                
def get_sig_dict_from_random_sampling(d,d_nb,method):
    '''get percentage significance through random sampling
    '''
    common_genes = set.intersection(set(d), set(d_nb))
    dict_sig = {}
    for g in common_genes:
        l_temp=[]
        for i in range(100):
            k_temp = {k:d[g][k] for k in random.sample(list(d[g]),k=len(d_nb[g]))}
            
            count=0
            for k,v in k_temp.items():
                if method=='wext' and v==0:
                    continue
                elif v<0.05:
                    count+=1
            
            l_temp.append(float(count)/len(k_temp))
        
        dict_sig[g] = np.mean(l_temp)
    
    return dict_sig

def plot_from_dict(d, d_mla,ax, annotation_threshold_y=-0.0,annotation_threhsold_x=1.5,title=''):
    l_mla_all = []
    l_sig_all = []
    
    l_mla_nonrare = []
    l_sig_nonrare = []
    
    l_mla_rare = []
    l_sig_rare = []
    
    for k,v in d.items():
        if k in d_mla:
            l_mla_all.append(d_mla[k])
            l_sig_all.append(v)
            
                
    ax.scatter(l_mla_all,l_sig_all, c='K',alpha=0.3)
    
    r,p = pearsonr(l_mla_all,l_sig_all)
    print('r:{}\nP:{}'.format(r,p))
    ax.text(0.85, 0.97, 'r = {}\nP = {:.3g}'.format(round(r,2), p), ha='left', va='top', fontsize=8,transform=ax.transAxes)
    
    for k,v in d.items():
        if v>annotation_threshold_y and d_mla[k]<annotation_threhsold_x:
            ax.annotate(k, (d_mla[k],v), ha='left',va='top',xytext=(d_mla[k]+0.08,v-0.01))
    
    ax.set_xlim(min(l_mla_all)-0.5,max(l_mla_all)+0.5)
    ax.set_ylim(-0.05,1.05)        
    labels = [str(int(round(float(item)*100))) for item in ax.get_yticks()]
    print(labels)
    
    ax.set_yticklabels(labels)
    ax.set_xlabel('Mutation Load Association (MLA)')
    ax.set_ylabel('Percentage of Significant Findings (P<0.05)')
    ax.set_title(title)# + '| {} total genes, {} rare, {} nonrare'.format(len(l_sig_all), len(l_sig_rare),len(l_sig_nonrare)) )
    
        
    
def plot_from_dict_arrow(d,d_nb, d_mla,ax, annotation_threshold_y=-0.0,annotation_threhsold_x=1.5,title='',c1='C3',c2='K'):
    '''plot with direction of arrow going from dict d to dict d_nb
    annotation thresholds define the points until which gene annotation 
    is done in the graph
    '''
    
    l_mla_all = []
    l_sig_all = []
    l_sig_nb = []
    
    
    for k,v in d.items():
        if k in d_mla and k in d_nb:
            l_mla_all.append(d_mla[k])
            l_sig_all.append(v)
            l_sig_nb.append(d_nb[k])
                
    ax.scatter(l_mla_all,l_sig_nb, c=c1,alpha=0.7)
    ax.scatter(l_mla_all,l_sig_all, c=c2,alpha=0.2)
    
    
    r,p = pearsonr(l_mla_all,l_sig_nb)
    print('r:{}\nP:{}'.format(r,p))
    ax.text(0.85, 0.97, 'r = {}\nP = {:.3g}'.format(round(r,2), p), ha='left', va='top', fontsize=8,transform=ax.transAxes)
    
    for k,v in d_nb.items():
        if v>annotation_threshold_y and d_mla[k]<annotation_threhsold_x:
            ax.annotate(k, (d_mla[k],v), ha='left',va='top',xytext=(d_mla[k]+0.08,v-0.01))
            
            
    for _mla,_all,_nb in zip(l_mla_all, l_sig_all,l_sig_nb):
        if _mla<annotation_threhsold_x and abs(_all-_nb)>0.01:
            if _all<_nb:
                ax.annotate("", xytext=(_mla, _all), xy=(_mla, _all+_nb-_all), arrowprops=dict(arrowstyle="->", alpha=0.2))
#                 ax.arrow(_mla,_all,0.0,_nb-_all, alpha=0.5,lw=2, length_includes_head=True)
#                 ax.quiver(_mla,_all,0.0,(_all-_nb), alpha=0.5,lw=0.1,headwidth=0.05, headlength=0.1,)
            else:
                ax.annotate("", xytext=(_mla, _all), xy=(_mla, _all+_nb-_all), arrowprops=dict(arrowstyle="->", alpha=0.2))
#                 ax.arrow(_mla,_all,0.0,_nb-_all, alpha=0.5,lw=2, length_includes_head=True)

#     for k,v in d.items():
#         if k in d_nb:
#             if d_mla[k]<annotation_threhsold_x:
#                 ax.arrow(d_mla[k], v, d_mla[k],d_nb[k])
            
        #     ax.arrow(l_mla_all[0],l_sig_all[0],l_sig_all[0]-l_sig_nb[0], l_mla_all[0]-l_mla_all[0],
#             shape='full', color='K', lw=1/2., length_includes_head=True, 
#             zorder=0)
    ax.set_xlim(min(l_mla_all)-0.5,max(l_mla_all)+0.5)
    ax.set_ylim(-0.05,1.05)        
    labels = [str(int(round(float(item)*100))) for item in ax.get_yticks()]

    ax.set_yticklabels(labels)        
    ax.set_xlabel('Mutation Load Association (MLA)')
    ax.set_ylabel('Percentage of Significant Findings (P<0.05)')
    ax.set_title(title)# + '| {} total genes, {} rare, {} nonrare'.format(len(l_sig_all), len(l_sig_rare),len(l_sig_nonrare)) )
    
    

## LOAD MUTEX VALUES

In [10]:
c= 'LUSC' # cancer type
t=20 #mutation threshold
methods = ['discover','fishers','wext']#'discover_strat','discover','discover_strat']#
inpath_mla = '../mutex_data/'

# inpath_sig = '../out_main/evaluation_results/significance_files/{}_t{}/'.format(c,t)


## Load MLA
MLA_infile = inpath_mla + 'MLA_ep_mutation_filtered_all_genes/{}_MLA_standardized.txt'.format(c)
with open(MLA_infile, 'r') as f:
    MLA = {line.split()[0]: float(line.split()[1]) for line in f.readlines()}
    
    
dict_infile, dict_infile_intact = set_input_file_paths(methods, c,t)

dict_cg_cg = load_cgcg_pairs(methods,dict_infile)
dict_cg_cg_nb = load_cgcg_pairs(methods,dict_infile_intact)

In [11]:
d_sig_m = get_percsig_dict(dict_cg_cg[methods[0]], methods[0])
d_sig_m

{'AFF3': 0.0,
 'AKAP9': 0.021505376344086023,
 'ANK1': 0.0,
 'APC': 0.010752688172043012,
 'ARID1A': 0.021505376344086023,
 'ARID1B': 0.0,
 'ARID2': 0.0,
 'ASXL1': 0.010752688172043012,
 'ATM': 0.0,
 'ATR': 0.0,
 'ATRX': 0.010752688172043012,
 'BCLAF1': 0.0,
 'BIRC6': 0.010752688172043012,
 'BRCA1': 0.010752688172043012,
 'BRCA2': 0.021505376344086023,
 'CAMTA1': 0.021505376344086023,
 'CDH10': 0.021505376344086023,
 'CDKN2A': 0.07526881720430108,
 'CNTNAP2': 0.043010752688172046,
 'COL2A1': 0.010752688172043012,
 'COL3A1': 0.0,
 'CREBBP': 0.010752688172043012,
 'CSMD3': 0.010752688172043012,
 'CTNNA2': 0.010752688172043012,
 'CTNND2': 0.010752688172043012,
 'CUL3': 0.0,
 'CUX1': 0.0,
 'DCC': 0.043010752688172046,
 'DROSHA': 0.010752688172043012,
 'EP300': 0.0,
 'EPHA3': 0.03225806451612903,
 'EPHA7': 0.0,
 'ERBB4': 0.010752688172043012,
 'FAM135B': 0.021505376344086023,
 'FAM47C': 0.043010752688172046,
 'FAT1': 0.053763440860215055,
 'FAT3': 0.043010752688172046,
 'FAT4': 0.0107526881

r:-0.5287764256937055
P:4.292038111398617e-08
['-20', '0', '20', '40', '60', '80', '100', '120']
r:-0.7967455627092122
P:3.179615215399538e-06
['-20', '0', '20', '40', '60', '80', '100', '120']
r:-0.5042076554027521
P:0.011992931711079556
r:-0.4104777463159826
P:3.971980240589068e-05
['-20', '0', '20', '40', '60', '80', '100', '120']
r:-0.587513720516016
P:0.002538491717425256
['-20', '0', '20', '40', '60', '80', '100', '120']
r:-0.5042076554027521
P:0.011992931711079556
r:-0.6148173672940616
P:4.3367343219935724e-11
['-20', '0', '20', '40', '60', '80', '100', '120']
r:-0.8195464553342003
P:9.624263670851674e-07
['-20', '0', '20', '40', '60', '80', '100', '120']
r:-0.5161087092899128
P:0.009830433744272983



In [16]:
## plot percentage significance

dict_methods = {'discover': 'DISCOVER',
               'discover_strat': 'DISCOVER Strat',
               'fishers': 'Fisher\'s Exact Test',
               'megsa': 'MEGSA',
               'memo': 'MEMO',
               'wext': 'WExT'}


sfx = '../results_main/evaluation_results/figures_v8_random_fig3ep_for paper/'

outpath_cgcg_nb = sfx

if not os.path.exists(outpath_cgcg_nb):
    os.makedirs(outpath_cgcg_nb)

#margins for annotation of genes within plot
val_x = -1.2
val_y = -0.1

fig, axes = plt.subplots(len(methods),3,figsize=(25,20)) #adjust figsize for the plot
letters = list(string.ascii_uppercase)[:len(methods)*3] #annotate figures alphabetiaclly
axiter=axes.flat #flatten axis to a list
for i,m in tqdm(enumerate(methods)):
    
    if m=='discover_strat':
        dict_sig = get_sig_dict(dict_cg_cg[m],m)
        dict_sig_discover = get_sig_dict(dict_cg_cg['discover'],'discover') #load discover to compare
        dict_sig_rand = get_sig_dict_from_random_sampling(dict_cg_cg[m],dict_cg_cg_nb[m],m) 
        dict_sig_rand_discover = get_sig_dict_from_random_sampling(dict_cg_cg['discover'],dict_cg_cg_nb['discover'],'discover')
        dict_sig_cg_cg_nb = get_sig_dict(dict_cg_cg_nb[m], m)

        #main plot
        ax=axiter[3*i]
        letter=letters[3*i]
        title = '{} COSMIC-COSMIC pairs'.format(dict_methods[m])
        plot_from_dict_arrow(dict_sig_discover,dict_sig,MLA,ax,title=title,c1='C0', annotation_threhsold_x=val_x, annotation_threshold_y=val_y)

        ax.text(-0.06, 1.1, letter, transform=ax.transAxes,
              fontsize=16, fontweight='bold', va='top', ha='right')

        #random sampled plot
        ax=axiter[3*i+1]
        letter=letters[3*i+1]
        title = '{} COSMIC-COSMIC pairs random'.format(dict_methods[m])
        plot_from_dict_arrow(dict_sig_rand_discover,dict_sig_rand,MLA,ax,title=title,c1='C0', annotation_threhsold_x=val_x, annotation_threshold_y=val_y)

        ax.text(-0.06, 1.1, letter, transform=ax.transAxes,
              fontsize=16, fontweight='bold', va='top', ha='right')
        
        #neighbors plot
        ax=axiter[3*i+2]
        letter=letters[3*i+2]
        title = '{} COSMIC-COSMIC neighbors'.format(dict_methods[m])
        plot_from_dict_arrow(dict_sig_rand,dict_sig_cg_cg_nb,MLA,ax,title=title,c2='C0', annotation_threhsold_x=val_x, annotation_threshold_y=val_y)

        ax.text(-0.06, 1.1, letter, transform=ax.transAxes,
              fontsize=16, fontweight='bold', va='top', ha='right')
    
    else:
        dict_sig = get_sig_dict(dict_cg_cg[m], m)
        dict_sig_rand = get_sig_dict_from_random_sampling(dict_cg_cg[m],dict_cg_cg_nb[m],m)
        dict_sig_cg_cg_nb = get_sig_dict(dict_cg_cg_nb[m], m)

        #main plot
        ax=axiter[3*i]
        letter=letters[3*i]
        title = '{} COSMIC-COSMIC pairs'.format(dict_methods[m])
        plot_from_dict(dict_sig,MLA,ax,title=title,annotation_threhsold_x=val_x,annotation_threshold_y=val_y)

        ax.text(-0.06, 1.1, letter, transform=ax.transAxes,
              fontsize=16, fontweight='bold', va='top', ha='right')

        #randomized plot
        ax=axiter[3*i+1]
        letter=letters[3*i+1]
        title = '{} COSMIC-COSMIC pairs random'.format(dict_methods[m])
        plot_from_dict(dict_sig_rand,MLA,ax,title=title,annotation_threhsold_x=val_x,annotation_threshold_y=val_y)

        ax.text(-0.06, 1.1, letter, transform=ax.transAxes,
              fontsize=16, fontweight='bold', va='top', ha='right')
        
        
        ax=axiter[3*i+2]
        letter=letters[3*i+2]
        title = '{} COSMIC-COSMIC neighbors'.format(dict_methods[m])
        plot_from_dict_arrow(dict_sig_rand,dict_sig_cg_cg_nb,MLA,ax,title=title, annotation_threhsold_x=val_x, annotation_threshold_y=val_y)

        ax.text(-0.06, 1.1, letter, transform=ax.transAxes,
              fontsize=16, fontweight='bold', va='top', ha='right')

plt.savefig(outpath_cgcg_nb+'{}_t{}_percsig_random_fig1.pdf'.format(c,t),format='pdf', bbox_inches='tight')
plt.show()

In [8]:
len(dict_cgcg_intact[m]),len(dict_sig_rand)

NameError: name 'dict_cgcg_intact' is not defined

## Testing individual genes - a separate section for testing select genes and their properties

In [27]:
binary_inpath = '../mutex_data/binary_matrices_all_genes_ep_mutation_filtered/'
binary_infile = binary_inpath + '{}_TML_binary_sm.txt'.format(c)

df = pd.read_csv(binary_infile, sep='\t', header=0,index_col=0)

genes=df.columns.tolist()

dict_gene_vs_patients = {}

for g in genes:
    dict_gene_vs_patients[g]=df.index[df[g] == 1].tolist()
    
dict_gene_vs_patients

{'A1BG': ['TCGA-AA-3663-01A',
  'TCGA-AA-3877-01A',
  'TCGA-AM-5821-01A',
  'TCGA-AZ-4315-01A',
  'TCGA-CA-6717-01A',
  'TCGA-CM-6162-01A',
  'TCGA-D5-6531-01A'],
 'A1CF': ['TCGA-A6-3809-01A',
  'TCGA-AA-3977-01A',
  'TCGA-AA-3984-01A',
  'TCGA-AG-A002-01A',
  'TCGA-CA-6717-01A',
  'TCGA-G4-6302-01A',
  'TCGA-G4-6588-01A'],
 'A2M': ['TCGA-A6-5660-01A',
  'TCGA-A6-6780-01A',
  'TCGA-A6-6781-01A',
  'TCGA-AA-3492-01A',
  'TCGA-AA-3502-01A',
  'TCGA-AA-3672-01A',
  'TCGA-AA-3693-01A',
  'TCGA-AA-3877-01A',
  'TCGA-AA-3949-01A',
  'TCGA-AA-3980-01A',
  'TCGA-AA-3984-01A',
  'TCGA-AA-A00N-01A',
  'TCGA-AD-6889-01A',
  'TCGA-AG-A002-01A',
  'TCGA-AZ-4616-01A',
  'TCGA-AZ-6598-01A',
  'TCGA-CA-6717-01A',
  'TCGA-CI-6624-01C',
  'TCGA-D5-5537-01A',
  'TCGA-D5-6927-01A',
  'TCGA-D5-6930-01A',
  'TCGA-DC-6158-01A',
  'TCGA-DC-6683-01A',
  'TCGA-EI-6882-01A',
  'TCGA-EI-6917-01A',
  'TCGA-F5-6814-01A',
  'TCGA-G4-6588-01A'],
 'A2ML1': ['TCGA-A6-3809-01A',
  'TCGA-A6-5661-01A',
  'TCGA-AA-3489-01A

In [79]:
test_genes = ['SMAD4','SMAD2','SMAD3']
list_vals=[]
m='discover'
for gx in test_genes[:]:
    nbx = len(dict_cg_cg_nb[m][gx])
    patientsx = len(dict_gene_vs_patients[gx])
    
    patientsy_union = set()
    patientsy_intersection= set()
    sig_count=0
    for i,gy in enumerate(dict_cg_cg_nb[m][gx]):
        if dict_cg_cg_nb[m][gx][gy]<0.05:
            sig_count+=1
        if i==0:
            patientsy_intersection = set(dict_gene_vs_patients[gy])
        else:
            patientsy_intersection.intersection(set(dict_gene_vs_patients[gy]))
        
        patientsy_union.update(dict_gene_vs_patients[gy])
    
        
    print(dict_cg_cg_nb[m][gx], nbx,sig_count, patientsx, len(patientsy_union),len(patientsy_intersection))
    list_vals.append([gx,nbx,sig_count, patientsx])#,len(patientsy_union),len(patientsy_intersection)])

cols = ['gene','neighbors', 'sig_nb','patients']
# 'union_neighbor_patients','intersection_neighbor_patients']

df_test = pd.DataFrame(list_vals,columns=cols)
df_test

{'SMAD3': 0.009352644537884577, 'SMAD2': 0.00651503435871381} 2 2 67 39 22
{'SMAD4': 0.00651503435871381, 'TP53': 0.004636756834414936, 'SMAD3': 0.7590496826372326, 'GNAS': 0.01564361270196663, 'TPR': 0.4294302253738141, 'MYO5A': 0.08204553488532945, 'PREX2': 0.03590973727325017, 'CREBBP': 0.05907299770492538, 'EP300': 0.2505268556523276, 'UBR5': 0.20733573541318845} 10 4 23 380 67
{'SMAD4': 0.009352644537884577, 'SMAD2': 0.7590496826372326, 'SETD2': 0.19724815847254215, 'FLNA': 0.3311510168363139} 4 1 22 126 67


,gene,neighbors,sig_nb,patients
0,SMAD4,2,2,67
1,SMAD2,10,4,23
2,SMAD3,4,1,22


In [80]:
s = set()
for i,gx in enumerate(test_genes[:]):
    s2=set()
    for j,gy in enumerate(dict_cg_cg_nb[m][gx]):
        s2.update(dict_gene_vs_patients[gy])
    print(len(s2))
    
    if i==0:
        s.update(s2)
    else:
        s.intersection_update(s2)

len(s)
    

39
380
126


26

In [81]:
for gx in test_genes:
    print(gx,':',list(dict_cg_cg_nb[m][gx]))

SMAD4 : ['SMAD3', 'SMAD2']
SMAD2 : ['SMAD4', 'TP53', 'SMAD3', 'GNAS', 'TPR', 'MYO5A', 'PREX2', 'CREBBP', 'EP300', 'UBR5']
SMAD3 : ['SMAD4', 'SMAD2', 'SETD2', 'FLNA']


2.995732273553991